In [92]:
# installs
! pip install pandas
! pip install petl


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [93]:
# imports
import pandas as pd
import petl as etl

1. Load CSV files to dataframes

In [94]:
csv_files_data = pd.read_csv('data_output/1_csv_files_dataframe.csv')
json_files_data = pd.read_csv('data_output/1_json_files_dataframe.csv')
sales_df = csv_files_data.append(json_files_data)


2. Prepare data for visual 1

In [95]:
# a) group data

sales_df_v1 = sales_df
sales_df_v1 = sales_df_v1.groupby(['branch', 'product']).sum().reset_index()

In [96]:
# b) convert to etl table and cut out not needed fields
sales_df_v1 = etl.fromdataframe(sales_df_v1)
sales_df_v1 = etl.transform.cutout(sales_df_v1, 'year', 'month', 'day', 'hour')

In [97]:
# c) lookup region and county

# branch list load
branch_list = etl.fromxlsx('branch_data/branch_list.xlsx')
branch_list = branch_list.rename({'branch_name': 'branch'})

# branch list join
sales_df_v1 = etl.outerjoin(sales_df_v1, branch_list)
sales_df_v1 = etl.transform.cutout(sales_df_v1, 'established_on')

In [98]:
# d) lookup product category

# product list load
product_list = etl.fromcsv('branch_data/products_list.csv')
product_list=etl.transform.cut(product_list, 'product', 'category')

# product list join
sales_df_v1 = etl.rightjoin(sales_df_v1, product_list, 'product')

In [99]:
# e) save to csv
etl.tocsv(sales_df_v1, 'data_output/2_visuals_pt1.csv')

3. Prepare data for visual 2

In [100]:
# a) group data

sales_df_v2 = sales_df
sales_df_v2 = sales_df_v2.groupby(['branch']).sum().reset_index()

In [101]:
# b) convert to etl table and cutout not needed columns
sales_df_v2 = etl.fromdataframe(sales_df_v2)
sales_df_v2 = etl.transform.cutout(sales_df_v2, 'year', 'month', 'day', 'hour')

In [102]:
# c) lookup region and county

# branch list load
branch_list = etl.fromxlsx('branch_data/branch_list.xlsx')

# branch list join
sales_df_v2 = etl.outerjoin(sales_df_v2, branch_list, lkey='branch', rkey='branch_name')
sales_df_v2 = etl.transform.cutout(sales_df_v2, 'established_on')

In [103]:
# d) save to csv
etl.tocsv(sales_df_v2, 'data_output/2_visuals_pt2.csv')

4. Prepare data for visual 3

In [104]:
# a) group and sum data
sales_df_v3 = sales_df
sales_df_v3 = sales_df_v3.groupby(['branch', 'year', 'month', 'day', 'hour']).sum().reset_index()

In [105]:
# b) group by branch and get average
sales_df_v3 = sales_df_v3.groupby(['branch']).mean().reset_index()

In [106]:
# c) coutout (drop) columns 
sales_df_v3 = sales_df_v3.drop(['year', 'month', 'day', 'hour', 'qty'], axis=1)

In [107]:
# d) save to csv
sales_df_v3.to_csv('data_output/2_visuals_pt3.csv')

5. Prepare data for visual 4

In [108]:
# a) group data by branch
sales_df_v4 = sales_df
sales_df_v4 = sales_df_v4.drop(['year', 'month', 'day', 'hour', 'qty'], axis=1)
sales_df_v4 = sales_df_v4.groupby('branch').sum().reset_index()

In [109]:
# b) get total expenses by branch
branch_expenses = pd.read_excel('branch_data/branch_expenses.xlsx')
branch_expenses = branch_expenses.drop(['year', 'month', 'region', 'county', 'established_on'], axis=1)
branch_expenses = branch_expenses.groupby('branch_name').sum().reset_index()

In [110]:
# c) sum expenses and drop useless columns
columns_list = ['operational_cost', 'staff_bonuses', 'misc_expenses', 'waste_cost']
branch_expenses['total_expenses'] = branch_expenses[columns_list].sum(axis=1)
branch_expenses = branch_expenses.drop(columns_list, axis=1)

In [111]:
# d) join tables and extract profit, drop sales and expenses

# convert to tables and join them
branch_expenses = etl.fromdataframe(branch_expenses)
sales_df_v4 = etl.fromdataframe(sales_df_v4)
sales_df_v4 = etl.outerjoin(sales_df_v4, branch_expenses, lkey='branch', rkey='branch_name')

# calculate profit column and cutout others
sales_df_v4 = etl.addfield(sales_df_v4, 'profit', lambda row: float(row.amount_in_gbp) - float(row.total_expenses))
sales_df_v4 = etl.cutout(sales_df_v4, 'amount_in_gbp', 'total_expenses')

In [112]:
# e) save to CSV
etl.tocsv(sales_df_v4, 'data_output/2_visuals_pt4.csv')